# Softmax, part 1

Task: practice using the `softmax` function.

**Why**: The softmax is a building block that is used throughout machine learning, statistics, data modeling, and even statistical physics. This activity is designed to get comfortable with how it works at a high and low level.

**Note**: Although "softmax" is the conventional name in machine learning, you may also see it called "soft *arg* max". The [Wikipedia article](https://en.wikipedia.org/w/index.php?title=Softmax_function&oldid=1065998663) has a good explanation.

## Setup

In [2]:
import torch
from torch import tensor
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

## Task

The following function defines `softmax` by using PyTorch built-in functionality.

In [3]:
def softmax_torch(x):
    '''Compute the softmax along the last axis, using PyTorch'''
    # axis=-1 means the last axis
    # This won't matter in this exercise, but it will matter when we get to batches of data.
    return torch.softmax(x, axis=-1)

Let's try it on an example tensor.

In [4]:
x = tensor([1., 2., 3.])
softmax_torch(x)

tensor([0.0900, 0.2447, 0.6652])

1. Start by playing with the interactive widget below. Describe the outputs when:

    1. All of the inputs are the same value. (Does it matter what the value is?)
      </br></br>The bar Graph Indicates same value for all outputs. It does not matter what the value is. </br></br>
    2. One input is much bigger than the others. 
      </br></br>The Bar graph gets skewed as one value is much greater and displayed longer, and other outputs are shortened to match the ratio. </br></br>
    
    3. One input is much smaller than the others.
    </br></br> Smallest input gets really short and others gets longer to match the ratio.</br></br>

Finally, describe the input that gives the largest possible value for output 1.

In [5]:
r = 2.0 # specify the range of the sliders
@widgets.interact(x0=(-r, r), x1=(-r, r), x2=(-r, r))
def show_softmax(x0, x1, x2):
    x = tensor([x0, x1, x2])
    xs = softmax_torch(x)
    plt.barh([2, 1, 0], xs)
    plt.xlim(0, 1)
    plt.yticks([2, 1, 0], ['output 0', 'output 1', 'output 2'])
    plt.ylabel("softmax(x)")
    return xs

interactive(children=(FloatSlider(value=0.0, description='x0', max=2.0, min=-2.0), FloatSlider(value=0.0, desc…

2. Fill in the following function to implement softmax yourself:

In [57]:
def softmax(xx):
    # Exponentiate x so all numbers are positive.
    expos = xx.exp()
    assert expos.min() >= 0
    # Normalize (divide by the sum).
    return expos / expos.sum()

3. Evaluate `softmax(x)` and verify (visually) that it is close to the `softmax_torch(x)` you evaluated above.

In [58]:
softmax(x)

tensor([0.0900, 0.2447, 0.6652])

4. Evaluate each of the following expressions (to make sure you understand what the output is), then evaluate `softmax_torch(__)` for each of the following expressions. Observe how each output relates to `softmax_torch(x)`. (Is it the same? Is it different? Why?)

- `x + 1`
- `x - 100`
- `x - x.max()`
- `x * 0.5`
- `x * 3.0`

In [59]:
softmax_torch(x+1)

tensor([0.0900, 0.2447, 0.6652])

In [60]:
softmax_torch(x-100)

tensor([0.0900, 0.2447, 0.6652])

In [61]:
softmax_torch(x - x.max())

tensor([0.0900, 0.2447, 0.6652])

In [62]:
softmax_torch(x*0.5)

tensor([0.1863, 0.3072, 0.5065])

In [63]:
softmax_torch(x*3.0)

tensor([0.0024, 0.0473, 0.9503])

## Analysis

Consider the following situation:

In [64]:
y2 = tensor([1., 0.,])
y3 = y2 - 1
y3

tensor([ 0., -1.])

In [65]:
y4 = y2 * 2
y4

tensor([2., 0.])

1. Are `softmax(y2)` and `softmax(y3)` the same or different? How could you tell without having to evaluate them?


Adding or subtracting to the exponential just cancels them out, so it won't change the output.

2. Are `softmax(y2)` and `softmax(y4)` the same or different? How could you tell without having to evaluate them?


They will be different because multiplication on exponential now has effect on the ouput.

## Optional Extension: Numerical Issues

### Task for Numerical Issues

5. *Numerical issues*. Assign `x2 = 50 * x`. Try `softmax(x2)` and observe that the result includes the dreaded `nan` -- "not a number". Something went wrong. **Evaluate the first mathematical operation in `softmax`** for this particularly problematic input. You should see another kind of abnormal value.

In [67]:
x2 = 50 * x
softmax(x2)

tensor([0., nan, nan])

In [72]:
x2.exp()

tensor([5.1847e+21,        inf,        inf])

6. *Fixing numerical issues*. Now try `softmax(x2 - 150.0)`. Observe that you now get valid numbers. Also observe how the constant we subtracted relates to the value of `x2`.

In [73]:
softmax(x2 - 150.0)

tensor([3.7835e-44, 1.9287e-22, 1.0000e+00])

7. Copy your `softmax` implementation to a new function, `softmax_stable`, and change it so that it subtracts `xx.max()` from `xx` before exponentiating. (Don't use any in-place operations; just use `xx - xx.max()`) Verify that `softmax_stable(x2)` now works, and obtains the same result as `softmax_torch(x2)`

In [74]:
def softmax(xx):
    # Exponentiate x so all numbers are positive.
    expos = (xx-xx.max()).exp()
    assert expos.min() >= 0
    # Normalize (divide by the sum).
    return expos / expos.sum()

In [75]:
softmax_torch(x2)

tensor([3.7835e-44, 1.9287e-22, 1.0000e+00])

### Analysis of Numerical Issues

3. Explain why `softmax(x2)` failed.

The function computes the exponential of each element in x2, and then divides the result by the sum of all exponentials in x2. However, when the values in x2 are too large, the exponentials can overflow, resulting in NaN (not a number) values.

4. Use your observations in \#1-2 above to explain why `softmax_stable` still gives the correct answer for `x` even though we changed the input.

It ensures that the softmax function can be computed correctly even for inputs with large or small values, by maximizing numerical stability while preserving the original order of the elements in each row. Therefore, the function can still give the correct answer for x even though we changed the input,

5. Explain why `softmax_stable` doesn't give us infinity or Not A Number anymore for `x2`.

If you subtract xx.max() you will not get any large numbers, so it won'y cause any troubles of getting inifinity or NaN

## Extension *optional*

Try to prove your observation in Analysis \#1 by symbolically simplifying the expression `softmax(logits + c)` and seeing if you can get `softmax(logits)`. Remember that `softmax(x) = exp(x) / exp(x).sum()` and `exp(a + b) = exp(a)exp(b)`.